https://datasets.imdbws.com/

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
sns.set()
from IPython.display import display
pd.options.display.max_columns = 35
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import kneighbors_graph
from google.colab import files
from IPython.display import Image, HTML
from google.colab.patches import cv2_imshow
from skimage import io
import urllib
import cv2

In [ ]:
uploaded = files.upload()

KeyboardInterrupt: ignored

In [ ]:
#tmdb = pd.read_csv('movies_additional_data.csv') # Additional dataset for pictures etc

In [ ]:
optimized_dtypes = {'tconst': 'object', 'title': 'object', 'region': 'category'}
titleFr = pd.concat([chunk[(chunk['region'] == "FR")] # on charge les filtres et on assemble le résultat
                         for chunk in pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', #boucle qui cheche les filtres sur le fichier
                                                  sep='\t',
                                                  usecols=[0,2,3], #colonnes a utiliser
                                                  names=optimized_dtypes.keys(), #colonnes renomée
                                                  iterator=True, #on peut iterer dessus sans mettre en RAM
                                                  chunksize=10000)]) #on divise en n lignes et on fait des fichiers txt
titleFr = titleFr.astype(dtype=optimized_dtypes)
titleFr['tconst'] = titleFr.tconst.drop_duplicates()

In [ ]:
optimized_dtypes = {'tconst': 'object', 'titleType': 'object', 'originalTitle': 'object', 'isAdult': 'uint8', 'startYear': 'int16', 'runtimeMinutes': 'int16', 'genres': 'category'}
film = pd.concat([chunk[(chunk['titleType'] == "movie") & (chunk["startYear"] != '\\N') & (chunk['runtimeMinutes'] != '\\N') & (chunk['genres'] != '\\N')]
                  for chunk in pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz',
                                               sep='\t',
                                               usecols=optimized_dtypes.keys(),
                                               iterator=True,
                                               chunksize=10000)])
film = film.astype(dtype=optimized_dtypes)
film = film[(film.startYear > 1960) & (film.runtimeMinutes > 80) & (film.runtimeMinutes < 210)]

In [ ]:
optimized_dtypes = {'tconst': 'object', 'averageRating': 'float16', 'numVotes': 'uint32'}
ratings = pd.concat([chunk[(chunk["averageRating"] != '\\N') & (chunk['numVotes'] != '\\N') & (chunk['numVotes'] > 1000) & (chunk['averageRating'] > 4)]
                      for chunk in pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz',
                                               sep='\t',
                                               usecols=optimized_dtypes.keys(),
                                               iterator=True,
                                               chunksize=10000)])
ratings = ratings.astype(dtype=optimized_dtypes)

In [ ]:
film_Rating = pd.merge(film, ratings, how="inner", on=["tconst"])
film_Rating_Fr = pd.merge(film_Rating, titleFr, how="inner", on=["tconst"])
imdb = pd.concat([film_Rating_Fr, film_Rating_Fr.genres.str.get_dummies(sep=',')
                  .astype('uint8')
                  .drop(columns=['Adult'])], axis=1).drop(columns=['genres','titleType','region'])
#tmdb = tmdb.drop(columns=['tmdb_id','backdrop_image_url','trailer_embed_url','complete_movie_embed_url'])
#tmdb = tmdb.rename(columns={'imdb_id':'tconst'})

<h1>KNN</h1>

In [ ]:
# Améliorer poids
X = imdb[['isAdult', 'startYear', 'runtimeMinutes',
       'averageRating', 'numVotes', 'Action', 'Adventure',
       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical', 'Mystery',
       'News', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']]

scale = StandardScaler().fit(X) 
X_scaled = scale.transform(X)

x_scaled = pd.DataFrame(X_scaled, columns=X.columns)

x_scaled.numVotes = x_scaled.numVotes * 0.8
x_scaled.startYear = x_scaled.startYear * 1.5
x_scaled.iloc[:,5:] = x_scaled.iloc[:,5:] * 1.5
x_scaled.averageRating = x_scaled.averageRating * 1.5

distanceKNN = NearestNeighbors(n_neighbors=6).fit(X_scaled)

try:
    predict = distanceKNN.kneighbors(X_scaled[imdb.title.str.lower() == input().lower()]) #Mettre le input en regex
    stop = 0
except ValueError:
    print("Le film n'est pas dans la séléction.")
    stop = 1

newFilm = pd.DataFrame(columns = imdb.columns) 

for i in range(6):
    if stop == 1: break
    newFilm = newFilm.append(imdb.iloc[predict[1][0][i],:])
    if i !=0 : print(np.array(newFilm.title)[i]) # Affiche que le nom des films
#newFilm[1:] #Debug

NameError: ignored

In [ ]:
image = pd.merge(newFilm,tmdb,how='left',on='tconst')

fig = plt.figure(figsize=(16, 10))
rows = 2
columns = 3
compteur = 1
for number in range(6):
    try:
        Image = urllib.request.urlopen(image.poster_url[number]) # Getting our picture from url
        fig.add_subplot(rows, columns, compteur) # Preparing subplots for better display
        arr = np.asarray(bytearray(Image.read()), dtype=np.uint8) # Creating an array of the picture
        img = cv2.imdecode(arr, -1) # Converting the array in image
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img) # Displaying
        cv2.waitKey(0) # Time before displaying
        plt.title(image.title[number]) # Assigning titles
        plt.axis('off') # Removing borders between sub-arrays
        compteur += 1
    except:
        print('Pas d\'affiche disponible pour',image.title[number]) # Handling error if the picture is not in the database

<h1>XGBoost</h1>

In [ ]:
X = imdb[['isAdult', 'startYear', 'runtimeMinutes',
       'averageRating', 'numVotes', 'Action', 'Adventure',
       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical', 'Mystery',
       'News', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']]

scale = StandardScaler().fit(X) 
X_scaled = scale.transform(X)


distanceKNN = NearestNeighbors(n_neighbors=6).fit(X_scaled)

boost = XGBClassifier().fit(X_scaled)
p_boost = boost.predict(Xtrain)